<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/notebook_templates/minimal_circuit_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

# Setup
(No need to change anything)

In [2]:
!pip install typing

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

In [3]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-q1gg317n
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-q1gg317n
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 20a44fe3a8022d353c9cc7c984a8fcab14552d1c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓

                           SCRIPT DEPRECATION WARNING                    

  
  This script, located at https://deb.nodesource.com/setup_X, used to
  install Node.js is deprecated now and will eventually be made inactive.

  Please visit the NodeSource distributions Github and follow the
  instructions to migrate your repo.
  https://github.com/nodesource/distributions

  The NodeSource Node.js Linux distribu

In [4]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [5]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [6]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [7]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [8]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

# Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [9]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# Import functions from repo

In [10]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

fatal: destination path 'ARENA_2.0' already exists and is not an empty directory.


In [11]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [12]:
import ioi_circuit_extraction as ioi_circuit_extraction

# Generate dataset with multiple prompts

In [ ]:
#@title Names list
names = [
    "Michael",
    "Christopher",
    "Jessica",
    "Matthew",
    "Ashley",
    "Jennifer",
    "Joshua",
    "Amanda",
    "Daniel",
    "David",
    "James",
    "Robert",
    "John",
    "Joseph",
    "Andrew",
    "Ryan",
    "Brandon",
    "Jason",
    "Justin",
    "Sarah",
    "William",
    "Jonathan",
    "Stephanie",
    "Brian",
    "Nicole",
    "Nicholas",
    "Anthony",
    "Heather",
    "Eric",
    "Elizabeth",
    "Adam",
    "Megan",
    "Melissa",
    "Kevin",
    "Steven",
    "Thomas",
    "Timothy",
    "Christina",
    "Kyle",
    "Rachel",
    "Laura",
    "Lauren",
    "Amber",
    "Brittany",
    "Danielle",
    "Richard",
    "Kimberly",
    "Jeffrey",
    "Amy",
    "Crystal",
    "Michelle",
    "Tiffany",
    "Jeremy",
    "Benjamin",
    "Mark",
    "Emily",
    "Aaron",
    "Charles",
    "Rebecca",
    "Jacob",
    "Stephen",
    "Patrick",
    "Sean",
    "Erin",
    "Jamie",
    "Kelly",
    "Samantha",
    "Nathan",
    "Sara",
    "Dustin",
    "Paul",
    "Angela",
    "Tyler",
    "Scott",
    "Katherine",
    "Andrea",
    "Gregory",
    "Erica",
    "Mary",
    "Travis",
    "Lisa",
    "Kenneth",
    "Bryan",
    "Lindsey",
    "Kristen",
    "Jose",
    "Alexander",
    "Jesse",
    "Katie",
    "Lindsay",
    "Shannon",
    "Vanessa",
    "Courtney",
    "Christine",
    "Alicia",
    "Cody",
    "Allison",
    "Bradley",
    "Samuel",
]

In [ ]:
def filter_names(names):
    return [name for name in names if len(model.tokenizer.tokenize(name)) == 1]
names = filter_names(names)

In [ ]:
import random

def make_prompts_list(names, template, num_sentences, num_targ_tokens):
    sentences = []
    generated_set = set() # Ensure none of the generated sentences are the same
    while len(sentences) < num_sentences:
        unique_names = random.sample(names, k=num_targ_tokens)
        temp_template = template
        sentence_dict = {}
        for i, name in enumerate(unique_names, start=1):
            temp_template = temp_template.replace(f"[S{i}]", name)
            sentence_dict[f'S{i}'] = name
        sentence_dict['text'] = temp_template
        if sentence_dict['text'] not in generated_set:
            generated_set.add(sentence_dict['text'])
            sentences.append(sentence_dict)
    return sentences

In [ ]:
class Dataset:
    def __init__(self, prompts, tokenizer, N, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = N
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["S1"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["S2"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if key != 'text']:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                    target_token = prompt[targ]
                else:
                    target_token = "Ġ" + prompt[targ]
                target_index = tokens.index(target_token)
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

Repalce io_tokens with correct answer (next, which is '5') and s_tokens with incorrect (current, which repeats)

In [ ]:
# template = "[S1] and [S2] went to the store. [S2] gave a drink to"
template = "Then, [S1] and [S2] went to the store, [S2] gave a drink to"
num_inputs = 30
num_targ_tokens = 2
prompts_list = make_prompts_list(names, template, num_inputs, num_targ_tokens)
dataset = Dataset(prompts_list, model.tokenizer, num_inputs, S1_is_first=False)

In [ ]:
dataset_2 = Dataset(prompts_list, model.tokenizer, num_inputs, S1_is_first=False)

# Early heads validation

In [13]:
from torch import Tensor
import torch as t
device = torch.device("cuda:0")

In [22]:
def generate_repeated_tokens(
	model: HookedTransformer,
	seq_len: int,
	batch: int = 1
) -> Float[Tensor, "batch 2*seq_len"]:
	'''
	Generates a sequence of repeated random tokens (no start token).
	'''
	rep_tokens_half = t.randint(0, model.cfg.d_vocab, (batch, seq_len), dtype=t.int64)
	rep_tokens = t.cat([rep_tokens_half, rep_tokens_half], dim=-1).to(device)
	return rep_tokens


def get_attn_scores(
	model: HookedTransformer,
	seq_len: int,
	batch: int,
	# head_type: Literal["duplicate", "prev", "induction"]
    head_type
):
	'''
	Returns attention scores for sequence of duplicated tokens, for every head.
	'''
	rep_tokens = generate_repeated_tokens(model, seq_len, batch)

	_, cache = model.run_with_cache(
		rep_tokens,
		return_type=None,
		names_filter=lambda name: name.endswith("pattern")
	)

	# Get the right indices for the attention scores

	if head_type == "duplicate":
		src_indices = range(seq_len)
		dest_indices = range(seq_len, 2 * seq_len)
	elif head_type == "prev":
		src_indices = range(seq_len)
		dest_indices = range(1, seq_len + 1)
	elif head_type == "induction":
		dest_indices = range(seq_len, 2 * seq_len)
		src_indices = range(1, seq_len + 1)
	else:
		raise ValueError(f"Unknown head type {head_type}")

	results = t.zeros(model.cfg.n_layers, model.cfg.n_heads, device="cuda", dtype=t.float32)
	for layer in range(model.cfg.n_layers):
		for head in range(model.cfg.n_heads):
			attn_scores: Float[Tensor, "batch head dest src"] = cache["pattern", layer]
			avg_attn_on_duplicates = attn_scores[:, head, dest_indices, src_indices].mean().item()
			results[layer, head] = avg_attn_on_duplicates

	return results


def plot_early_head_validation_results(seq_len: int = 50, batch: int = 50):
	'''
	Produces a plot that looks like Figure 18 in the paper.
	'''
	head_types = ["duplicate", "prev", "induction"]

	results = t.stack([
		get_attn_scores(model, seq_len, batch, head_type=head_type)
		for head_type in head_types
	])

	imshow(
		results,
		facet_col=0,
		facet_labels=[
			f"{head_type.capitalize()} token attention prob.<br>on sequences of random tokens"
			for head_type in head_types
		],
		labels={"x": "Head", "y": "Layer"},
		width=1300,
	)


model.reset_hooks()
plot_early_head_validation_results()

TypeError: ignored

# Ablate the model and compare with original

In [ ]:
# from ioi_dataset import NAMES, IOIDataset

# N = 25
# ioi_dataset = IOIDataset(
#     prompt_type="mixed",
#     N=N,
#     tokenizer=model.tokenizer,
#     prepend_bos=False,
#     seed=1,
#     # device=str(device)
# )
# abc_dataset = ioi_dataset.gen_flipped_prompts("ABB->XYZ, BAB->XYZ")

In [ ]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [ ]:
CIRCUIT = {
    "name mover": [(9, 9), (10, 0), (9, 6)],
    "backup name mover": [(10, 10), (10, 6), (10, 2), (10, 1), (11, 2), (9, 7), (9, 0), (11, 9)],
    "negative name mover": [(10, 7), (11, 10)],
    "s2 inhibition": [(7, 3), (7, 9), (8, 6), (8, 10)],
    # "induction": [(5, 5), (5, 8), (5, 9), (6, 9)],
    # "duplicate token": [(0, 1), (0, 10), (3, 0)],
    # "previous token": [(2, 2), (4, 11)],
}

SEQ_POS_TO_KEEP = {
    "name mover": "end",
    "backup name mover": "end",
    "negative name mover": "end",
    "s2 inhibition": "end",
    # "induction": "S2",
    # "duplicate token": "S2",
    # "previous token": "S1+1",
}

We can set the mean ablation to be over the same dataset, though with some issues

In [ ]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model(dataset.toks)

print(f"Average logit difference (IOI dataset, using entire model): {logits_to_ave_logit_diff_2(ioi_logits_original, dataset):.4f}")
print(f"Average logit difference (IOI dataset, only using circuit): {logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset):.4f}")

Average logit difference (IOI dataset, using entire model): 2.5075
Average logit difference (IOI dataset, only using circuit): 2.2445


## Test circuit without main name movers

In [ ]:
CIRCUIT = {
    # "name mover": [(9, 9), (10, 0), (9, 6)],
    "backup name mover": [(10, 10), (10, 6), (10, 2), (10, 1), (11, 2), (9, 7), (9, 0), (11, 9)],
    "negative name mover": [(10, 7), (11, 10)],
    "s2 inhibition": [(7, 3), (7, 9), (8, 6), (8, 10)],
    # "induction": [(5, 5), (5, 8), (5, 9), (6, 9)],
    # "duplicate token": [(0, 1), (0, 10), (3, 0)],
    # "previous token": [(2, 2), (4, 11)],
}

SEQ_POS_TO_KEEP = {
    # "name mover": "end",
    "backup name mover": "end",
    "negative name mover": "end",
    "s2 inhibition": "end",
    # "induction": "S2",
    # "duplicate token": "S2",
    # "previous token": "S1+1",
}

In [ ]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model(dataset.toks)

print(f"Average logit difference (IOI dataset, using entire model): {logits_to_ave_logit_diff_2(ioi_logits_original, dataset):.4f}")
print(f"Average logit difference (IOI dataset, only using circuit): {logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset):.4f}")

Average logit difference (IOI dataset, using entire model): 2.5075
Average logit difference (IOI dataset, only using circuit): 1.9428


## Test circuit without backup name movers

In [ ]:
CIRCUIT = {
    "name mover": [(9, 9), (10, 0), (9, 6)],
    # "backup name mover": [(10, 10), (10, 6), (10, 2), (10, 1), (11, 2), (9, 7), (9, 0), (11, 9)],
    "negative name mover": [(10, 7), (11, 10)],
    "s2 inhibition": [(7, 3), (7, 9), (8, 6), (8, 10)],
    # "induction": [(5, 5), (5, 8), (5, 9), (6, 9)],
    # "duplicate token": [(0, 1), (0, 10), (3, 0)],
    # "previous token": [(2, 2), (4, 11)],
}

SEQ_POS_TO_KEEP = {
    "name mover": "end",
    # "backup name mover": "end",
    "negative name mover": "end",
    "s2 inhibition": "end",
    # "induction": "S2",
    # "duplicate token": "S2",
    # "previous token": "S1+1",
}

In [ ]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model(dataset.toks)

print(f"Average logit difference (IOI dataset, using entire model): {logits_to_ave_logit_diff_2(ioi_logits_original, dataset):.4f}")
print(f"Average logit difference (IOI dataset, only using circuit): {logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset):.4f}")

Average logit difference (IOI dataset, using entire model): 2.5075
Average logit difference (IOI dataset, only using circuit): 2.0672


## Test circuit without all name movers

In [ ]:
CIRCUIT = {
    # "name mover": [(9, 9), (10, 0), (9, 6)],
    # "backup name mover": [(10, 10), (10, 6), (10, 2), (10, 1), (11, 2), (9, 7), (9, 0), (11, 9)],
    # "negative name mover": [(10, 7), (11, 10)],
    "s2 inhibition": [(7, 3), (7, 9), (8, 6), (8, 10)],
    # "induction": [(5, 5), (5, 8), (5, 9), (6, 9)],
    # "duplicate token": [(0, 1), (0, 10), (3, 0)],
    # "previous token": [(2, 2), (4, 11)],
}

SEQ_POS_TO_KEEP = {
    # "name mover": "end",
    # "backup name mover": "end",
    # "negative name mover": "end",
    "s2 inhibition": "end",
    # "induction": "S2",
    # "duplicate token": "S2",
    # "previous token": "S1+1",
}

In [ ]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model(dataset.toks)

print(f"Average logit difference (IOI dataset, using entire model): {logits_to_ave_logit_diff_2(ioi_logits_original, dataset):.4f}")
print(f"Average logit difference (IOI dataset, only using circuit): {logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset):.4f}")

Average logit difference (IOI dataset, using entire model): 2.5075
Average logit difference (IOI dataset, only using circuit): 0.4925


This shows name movers are very important.

## Test a wrong circuit to see that the logit diff doesn't match original

We keep only 4 heads, so that means we knock out most of the circuit

In [ ]:
WRONG_CIRCUIT = {
    "name mover": [(2,2)],
    "backup name mover": [(1, 1)],
    "negative name mover": [],
    "s2 inhibition": [],
    "induction": [(4,4)],
    "duplicate token": [],
    "previous token": [(8,3)],
}

In [ ]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

ioi_logits_original, ioi_cache = model.run_with_cache(ioi_dataset.toks)

model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=abc_dataset, circuit=WRONG_CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model(ioi_dataset.toks)

print(f"Average logit difference (IOI dataset, using entire model): {logits_to_ave_logit_diff_2(ioi_logits_original):.4f}")
print(f"Average logit difference (IOI dataset, only using circuit): {logits_to_ave_logit_diff_2(ioi_logits_minimal):.4f}")

Average logit difference (IOI dataset, using entire model): 2.8052
Average logit difference (IOI dataset, only using circuit): -0.7422


# Plot minimality scores

In [ ]:
%%capture
!pip install circuitsvis

Write a function that converts into this:

In [ ]:
K_FOR_EACH_COMPONENT = {
    (9, 9): set(),
    (10, 0): {(9, 9)},
    (9, 6): {(9, 9), (10, 0)},
    (10, 7): {(11, 10)},
    (11, 10): {(10, 7)},
    (8, 10): {(7, 9), (8, 6), (7, 3)},
    (7, 9): {(8, 10), (8, 6), (7, 3)},
    (8, 6): {(7, 9), (8, 10), (7, 3)},
    (7, 3): {(7, 9), (8, 10), (8, 6)},
    (5, 5): {(5, 9), (6, 9), (5, 8)},
    (5, 9): {(11, 10), (10, 7)},
    (6, 9): {(5, 9), (5, 5), (5, 8)},
    (5, 8): {(11, 10), (10, 7)},
    (0, 1): {(0, 10), (3, 0)},
    (0, 10): {(0, 1), (3, 0)},
    (3, 0): {(0, 1), (0, 10)},
    (4, 11): {(2, 2)},
    (2, 2): {(4, 11)},
    (11, 2): {(9, 9), (10, 0), (9, 6)},
    (10, 6): {(9, 9), (10, 0), (9, 6), (11, 2)},
    (10, 10): {(9, 9), (10, 0), (9, 6), (11, 2), (10, 6)},
    (10, 2): {(9, 9), (10, 0), (9, 6), (11, 2), (10, 6), (10, 10)},
    (9, 7): {(9, 9), (10, 0), (9, 6), (11, 2), (10, 6), (10, 10), (10, 2)},
    (10, 1): {(9, 9), (10, 0), (9, 6), (11, 2), (10, 6), (10, 10), (10, 2), (9, 7)},
    (11, 9): {(9, 9), (10, 0), (9, 6), (9, 0)},
    (9, 0): {(9, 9), (10, 0), (9, 6), (11, 9)},
}

In [ ]:
from solutions import *

def get_score(
	model: HookedTransformer,
	ioi_dataset: IOIDataset,
	abc_dataset: IOIDataset,
	K: Set[Tuple[int, int]],
	C: Dict[str, List[Tuple[int, int]]],
) -> float:
	'''
	Returns the value F(C \ K), where F is the logit diff, C is the
	core circuit, and K is the set of circuit components to remove.
	'''
	C_excl_K = {k: [head for head in v if head not in K] for k, v in C.items()}
	model = ioi_circuit_extraction.add_mean_ablation_hook(model, abc_dataset, C_excl_K, SEQ_POS_TO_KEEP)
	logits = model(ioi_dataset.toks)
	score = logits_to_ave_logit_diff_2(logits, ioi_dataset).item()

	return score

def get_minimality_score(
		model: HookedTransformer,
		ioi_dataset: IOIDataset,
		abc_dataset: IOIDataset,
		v: Tuple[int, int],
		K: Set[Tuple[int, int]],
		C: Dict[str, List[Tuple[int, int]]] = CIRCUIT,
	) -> float:
		'''
		Returns the value | F(C \ K_union_v) - F(C | K) |, where F is
		the logit diff, C is the core circuit, K is the set of circuit
		components to remove, and v is a head (not in K).
		'''
		assert v not in K
		K_union_v = K | {v}
		C_excl_K_score = get_score(model, ioi_dataset, abc_dataset, K, C)
		C_excl_Kv_score = get_score(model, ioi_dataset, abc_dataset, K_union_v, C)

		return abs(C_excl_K_score - C_excl_Kv_score)

def get_all_minimality_scores(
		model: HookedTransformer,
		ioi_dataset: IOIDataset = ioi_dataset,
		abc_dataset: IOIDataset = abc_dataset,
		k_for_each_component: Dict = K_FOR_EACH_COMPONENT
	) -> Dict[Tuple[int, int], float]:
		'''
		Returns dict of minimality scores for every head in the model (as
		a fraction of F(M), the logit diff of the full model).

		Warning - this resets all hooks at the end (including permanent).
		'''
		# Get full circuit score F(M), to divide minimality scores by
		model.reset_hooks(including_permanent=True)
		logits = model(ioi_dataset.toks)
		full_circuit_score = logits_to_ave_logit_diff_2(logits, ioi_dataset).item()

		# Get all minimality scores, using the `get_minimality_score` function
		minimality_scores = {}
		for v, K in tqdm(k_for_each_component.items()):
			score = get_minimality_score(model, ioi_dataset, abc_dataset, v, K)
			minimality_scores[v] = score / full_circuit_score

		model.reset_hooks(including_permanent=True)

		return minimality_scores

In [ ]:
minimality_scores = get_all_minimality_scores(model)

In [ ]:
def plot_minimal_set_results(minimality_scores: Dict[Tuple[int, int], float]):
    '''
    Plots the minimality results, in a way resembling figure 7 in the paper.

    minimality_scores:
        Dict with elements like (9, 9): minimality score for head 9.9 (as described
        in section 4.2 of the paper)
    '''

    CIRCUIT_reversed = {head: k for k, v in CIRCUIT.items() for head in v}
    colors = [CIRCUIT_reversed[head].capitalize() + " head" for head in minimality_scores.keys()]
    color_sequence = [px.colors.qualitative.Dark2[i] for i in [0, 1, 2, 5, 3, 6]] + ["#BAEA84"]

    bar(
        list(minimality_scores.values()),
        x=list(map(str, minimality_scores.keys())),
        labels={"x": "Attention head", "y": "Change in logit diff", "color": "Head type"},
        color=colors,
        template="ggplot2",
        color_discrete_sequence=color_sequence,
        bargap=0.02,
        yaxis_tickformat=".0%",
        legend_title_text="",
        title="Plot of minimality scores (as percentages of full model logit diff)",
        width=800,
        hovermode="x unified"
    )

plot_minimal_set_results(minimality_scores)